In [52]:
import os
import pickle
import tsplib95
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [53]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from pkl {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G.to_undirected()

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    # print(f"Loading {name} from tsplib {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    # remove redundant edges
    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    # remove self loops
    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    # save as pkl file
    with open(graph_file_path, "wb") as file:
        print(f"Saving {name} to pkl {graph_file_path}")
        pickle.dump(G, file)

    return G

In [54]:
tsp_dataset_file_path = os.path.join("tsp_dataset", "tsp_dataset.csv")
print(f"Loading tsp dataset from {tsp_dataset_file_path}")
tsp_dataset = pd.read_csv(tsp_dataset_file_path)

Loading tsp dataset from tsp_dataset\tsp_dataset.csv


In [55]:
def get_opt_tour_length(name):
    return tsp_dataset[tsp_dataset["name"] == name]["opt_tour_length"].values[0]

In [86]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [87]:
def get_apx_ratio(G: nx.graph, tour: list):
    return get_tour_length(G, tour) / get_opt_tour_length(G.name)

In [88]:
tsp_dataset

,name,dimension,opt_tour_length,type,comment,edge_weight_type,edge_weight_format
0,burma14,14,3323,TSP,14-Staedte in Burma (Zaw Win),GEO,FUNCTION
1,ulysses16,16,6859,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
2,gr17,17,2085,TSP,17-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
3,gr21,21,2707,TSP,21-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
4,ulysses22,22,7013,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
...,...,...,...,...,...,...,...
91,u2319,2319,234256,TSP,Drilling problem (Reinelt),EUC_2D,NaN
92,pr2392,2392,378032,TSP,2392-city problem (Padberg/Rinaldi),EUC_2D,NaN
93,pcb3038,3038,137694,TSP,Drilling problem (Junger/Reinelt),EUC_2D,NaN
94,fnl4461,4461,182566,TSP,Die 5 neuen Laender Deutschlands (Ex-DDR) (Bac...,EUC_2D,NaN


In [90]:
for name in tsp_dataset["name"]:
    G = get_tsp_graph(name)
    
    tour = christofides(G)
    tour_length = get_tour_length(G, tour)
    apx_ratio = get_apx_ratio(G, tour)
    
    problem = tsplib95.load(os.path.join("tsp_dataset", f"{name}.tsp"))
    
    print(name, tour_length, apx_ratio)
    print(tour)
    
    print()
    break

burma14 3606 1.085164008426121
[1, 2, 3, 14, 12, 6, 5, 4, 7, 13, 8, 10, 9, 11, 1]

